# A two stage optimziation problem generator

In [1]:
scenario_data = {
    "T": range(1, 6),  # Time periods 1 to 5
    "scenarios": {
        "scenario1": {
            "probability": 0.3,
            "parameters": {
                "W": {1: 10, 2: 15, 3: 20, 4: 25, 5: 30},
                "DR": {0: 5, 1: 5, 2: 6, 3: 7, 4: 8},
                "P": {1: 3, 2: 3, 3: 4, 4: 4, 5: 5},
                "CR": {1: 2, 2: 2, 3: 2, 4: 3, 5: 3},
                "ETc": {1: 1, 2: 1, 3: 1, 4: 2, 5: 2},
                "DP": {1: 1, 2: 1, 3: 2, 4: 2, 5: 3},
            },
        },
        "scenario2": {
            "probability": 0.7,
            "parameters": {
                "W": {1: 12, 2: 18, 3: 22, 4: 28, 5: 32},
                "DR": {0: 6, 1: 7, 2: 7, 3: 8, 4: 9},
                "P": {1: 4, 2: 4, 3: 5, 4: 5, 5: 6},
                "CR": {1: 3, 2: 3, 3: 3, 4: 4, 5: 4},
                "ETc": {1: 2, 2: 2, 3: 2, 4: 3, 5: 3},
                "DP": {1: 2, 2: 2, 3: 3, 4: 3, 5: 4},
            },
        },
        # More scenarios can be added similarly
    }
}

# The fixed parameter 'i' for the time step difference between stages
i = 1  # For example, a one time period lag between stages


In [2]:
!pip install pyomo
!pip install swiglpk



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip


In [5]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, ConstraintList, NonNegativeReals, maximize, SolverFactory

from pyomo.opt import SolverFactory

# Sample scenario data
scenario_data = {
    "T": range(1, 6),  # Time periods 1 to 5
    "scenarios": {
        "scenario1": {
            "probability": 0.3,
            "parameters": {
                "W": {1: 10, 2: 15, 3: 20, 4: 25, 5: 30},
                "DR": {0: 5, 1: 5, 2: 6, 3: 7, 4: 8},
                "P": {1: 3, 2: 3, 3: 4, 4: 4, 5: 5},
                "CR": {1: 2, 2: 2, 3: 2, 4: 3, 5: 3},
                "ETc": {1: 1, 2: 1, 3: 1, 4: 2, 5: 2},
                "DP": {1: 1, 2: 1, 3: 2, 4: 2, 5: 3},
            },
        },
        "scenario2": {
            "probability": 0.7,
            "parameters": {
                "W": {1: 12, 2: 18, 3: 22, 4: 28, 5: 32},
                "DR": {0: 6, 1: 7, 2: 7, 3: 8, 4: 9},
                "P": {1: 4, 2: 4, 3: 5, 4: 5, 5: 6},
                "CR": {1: 3, 2: 3, 3: 3, 4: 4, 5: 4},
                "ETc": {1: 2, 2: 2, 3: 2, 4: 3, 5: 3},
                "DP": {1: 2, 2: 2, 3: 3, 4: 3, 5: 4},
            },
        },
        # Additional scenarios can be added here
    }
}

i = 1  # The fixed parameter 'i' for the time step difference between stages

T = scenario_data["T"]
S = list(scenario_data["scenarios"].keys())

# Create a ConcreteModel instance
model = ConcreteModel()

# Define decision variables
model.I = Var(T, S, within=NonNegativeReals)  # Irrigation
model.D = Var(T, S, within=NonNegativeReals)  # Drainage

# Objective function
def objective_rule(model):
    return sum(scenario_data["scenarios"][s]["probability"] * model.I[t+i, s] for s in S for t in T if t+i in T)
model.obj = Objective(rule=objective_rule, sense=maximize)

# Constraints
model.constraints = ConstraintList()

# Adding water balance constraints for both stages
for t in T:
    for s in S:
        params = scenario_data["scenarios"][s]["parameters"]

        # Constraint for stage 1
        if t-1 in T:  # Check if t-1 is within the range of T
            model.constraints.add(params["W"][t-1] + params["P"][t] + model.I[t, s] + params["CR"][t] - (model.D[t, s] + params["DR"][t-1] + params["ETc"][t] + params["DP"][t]) == params["W"][t])

        # Constraint for stage 2
        if t+i in T:  # Ensure t+i is within the range of T
            model.constraints.add(params["W"][t+i-1] + params["P"][t+i] + model.I[t+i, s] + params["CR"][t+i] - (model.D[t+i, s] + params["DR"][t+i-1] + params["ETc"][t+i] + params["DP"][t+i]) == params["W"][t+i])

def probability_sum_rule(model):
    tolerance = 1e-5
    prob_sum = sum(scenario_data["scenarios"][s]["probability"] for s in S)
    return inequality(1-tolerance, prob_sum, 1+tolerance)
model.probability_sum_constraint = Constraint(rule=probability_sum_rule)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Access the solution
solution_I = {t: {s: model.I[t, s].value for s in S if t+i in T} for t in T if t+i in T}
solution_D = {t: {s: model.D[t, s].value for s in S if t+i in T} for t in T if t+i in T}

# Print the solution
print("Irrigation:", solution_I)
print("Drainage:", solution_D)


Irrigation: {1: {'scenario1': None, 'scenario2': None}, 2: {'scenario1': None, 'scenario2': None}, 3: {'scenario1': None, 'scenario2': None}, 4: {'scenario1': None, 'scenario2': None}}
Drainage: {1: {'scenario1': None, 'scenario2': None}, 2: {'scenario1': None, 'scenario2': None}, 3: {'scenario1': None, 'scenario2': None}, 4: {'scenario1': None, 'scenario2': None}}
